In [1]:
import numpy as np
import gurobipy as grb
import analyzer
import linear_solver
from gurobipy import *

import sys
sys.path.insert(0, '../ELINA/python_interface/')
from elina_box import *
from elina_interval import *
from elina_abstract0 import *
from elina_manager import *
from elina_dimension import *
from elina_scalar import *
from elina_interval import *
from elina_linexpr0 import *
from elina_lincons0 import *
import ctypes
from ctypes.util import find_library
import time

In [237]:
def linear_solver(inf, sup, weights, biases, numlayers,output):

    opt_model = Model("mip1")
    max_model = Model("max")
    
  
    for layer in range(numlayers):
        #j is the index of the current output node optimized
      
        if (layer!=layerno):
            layer_weights = nn.weights[layer+1]
            layer_biases = nn.biases[layer+1]
            #print("biases_layer_{}".format(layer))
            #print(layer_biases)

            np.ascontiguousarray(layer_weights, dtype=np.double)
            np.ascontiguousarray(layer_biases, dtype=np.double)

            num_out_pixels = len(layer_biases)
            num_in_pixels = len(layer_weights[0])
            
        else:
            num_out_pixels = 10
            num_in_pixels = 10
        set_I = range(num_in_pixels) 
        set_J = range(num_out_pixels)
        #print(inf)
        #print(sup)
  
        

        set_optimize_I = [i for i in set_I if inf[layer][i]<0 and sup[layer][i]>0] #boxes optimized by linear program
        set_zeros_I = [i for i in set_I if inf[layer][i]<=0 and sup[layer][i]<=0]  #boxes taken to [0,0] by relu 
        set_leave_I = [i for i in set_I if inf[layer][i]>0 and sup[layer][i]>0] #boxes left as before relu    
        
        lambda_ = [0]*num_in_pixels
        for i in set_optimize_I:
            lambda_[i]=(sup[layer][i]/(sup[layer][i]-inf[layer][i]))

        mu = [0]*num_in_pixels
        for i in set_optimize_I:
            mu[i]=((-1)*(sup[layer][i]*inf[layer][i])/(sup[layer][i]-inf[layer][i]))
        

        #---------------------------------------
        #MINIMIZATION

        
        #opt_model.setParam('OutputFlag', False)
        h_i_vars  = opt_model.addVars(num_in_pixels, lb=inf[layer], ub=sup[layer], name='h{0},'.format(layer))

        if(layer):
            constraints = {i: opt_model.addConstr(lhs=objective[i],sense=GRB.EQUAL,rhs=h_i_vars[i],
                                                 name="prev_constraint_{0}".format(i)) for i in set_I}
        
        relu_h_i_vars  = opt_model.addVars(num_in_pixels, name='relu_h{0},'.format(layer))
        
    
        
            # Set objective sum over i wji*relu_h_is
        if (layer!=layerno):
            objective = [grb.quicksum(layer_weights[j][i] * relu_h_i_vars[i] for i in set_I) + layer_biases[j]
                         for j in set_J]
        else: 
            objective = [relu_h_i_vars[i]*1 for i in set_I]
        
   

            # for minimization


       

        constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.LESS_EQUAL,
                                               rhs=lambda_[i] * h_i_vars[i] + mu[i], 
                                               name="constraint_lq_{0},{1}".format(layer,i))
                       for i in set_optimize_I}

            # >= constraints
        constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=0, 
                                               name="constraint_zero_{0},{1}".format(layer,i))
                       for i in set_optimize_I}

        # >= constraints
        constraints = {i :opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=h_i_vars[i], 
                                              name="constraint_grq_{0},{1}".format(layer,i)) for i in set_optimize_I}

        # == constraints
        constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=h_i_vars[i],
                                               name="constraint_{0},{1}".format(layer,i)) 
                       for i in set_leave_I}

        constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=0,
                                               name="constraint_{0},{1}".format(layer,i)) 
                      for i in set_zeros_I}
        
        
       # print([objective[0].getCoeff(i) for i in range(objective[0].size())]) 
       # print([objective[0].getVar(i) for i in range(objective[0].size())])

       # print(objective[0].getConstant())

        


        #-------------------------------------------------
        #MAXIMIZATION  

        h_i_vars  = max_model.addVars(num_in_pixels, lb=inf[layer], ub=sup[layer], name='h{0},'.format(layer))
       
        if(layer):
            constraints = {i: max_model.addConstr(lhs=objective_2[i],sense=GRB.EQUAL,rhs=h_i_vars[i],
                                   name="prev_constraint_{0}".format(i)) for i in set_I}

        relu_h_i_vars  = max_model.addVars(num_in_pixels, name='relu_h{0},'.format(layer))
       


            # Set objective sum over i wji*relu_h_is
        if (layer!=layerno):
            objective_2 = [grb.quicksum(layer_weights[j][i] * relu_h_i_vars[i] for i in set_I) + layer_biases[j]
                         for j in set_J]
        else:
            objective_2 = [relu_h_i_vars[i]*1 for i in set_I]



        constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.LESS_EQUAL,
                                               rhs=lambda_[i] * h_i_vars[i] + mu[i], 
                                               name="constraint_lq_{0},{1}".format(layer,i))
                       for i in set_optimize_I}

            # >= constraints
        constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=0, 
                                               name="constraint_zero_{0},{1}".format(layer,i))
                       for i in set_optimize_I}

        # >= constraints
        constraints = {i :max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=h_i_vars[i], 
                                              name="constraint_grq_{0},{1}".format(layer,i)) for i in set_optimize_I}

        # == constraints
        constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=h_i_vars[i],
                                               name="constraint_{0},{1}".format(layer,i)) 
                       for i in set_leave_I}

        constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=0,
                                               name="constraint_{0},{1}".format(layer,i)) 
                      for i in set_zeros_I}
        
        
  
    opt_model.setObjective(objective[output], GRB.MINIMIZE)
    opt_model.setParam('OutputFlag', False)
    opt_model.write('out1_min.mps')
    opt_model.optimize()
   # opt_model.computeIIS()
    
  #  opt_model.write('out1.ilp')
    h_j_min = opt_model.objVal

    max_model.setObjective(objective_2[output],GRB.MAXIMIZE)
    max_model.setParam('OutputFlag',False)
    max_model.optimize()
    #max_model.computeIIS()
    max_model.write('out_max.mps')
    #max_model.write('out1.ilp')

    h_j_max = max_model.objVal

    return h_j_min, h_j_max

In [240]:
with open('../mnist_nets/mnist_relu_3_20.txt', 'r') as netfile:
    netstring = netfile.read()

with open('../mnist_images/img1.txt', 'r') as specfile:
    specstring = specfile.read()

x0_low, x0_high = analyzer.parse_spec(specstring)
LB_N0, UB_N0 = analyzer.get_perturbed_image(x0_low,0.01)

In [241]:
nn = analyzer.parse_net(netstring)
nn.ffn_counter=0


num_pixels = len(LB_N0)
man = elina_box_manager_alloc()
itv = elina_interval_array_alloc(num_pixels)
for i in range(num_pixels):
    elina_interval_set_double(itv[i],LB_N0[i],UB_N0[i]) # specify itv[i]=[lb,ub] 
### Perform the box approximation for the first layer
## construct input abstraction
element = elina_abstract0_of_box(man, 0, num_pixels, itv)
elina_interval_array_free(itv,num_pixels)
myinf = []
mysup = []

for layerno in range(nn.numlayer):
    weights = nn.weights[nn.ffn_counter]
    biases = nn.biases[nn.ffn_counter]
    dims = elina_abstract0_dimension(man,element)
    num_in_pixels = dims.intdim + dims.realdim
    num_out_pixels = len(weights)

    dimadd = elina_dimchange_alloc(0,num_out_pixels)    
    for i in range(num_out_pixels):
        dimadd.contents.dim[i] = num_in_pixels
    elina_abstract0_add_dimensions(man, True, element, dimadd, False)
    elina_dimchange_free(dimadd)
    np.ascontiguousarray(weights, dtype=np.double)
    np.ascontiguousarray(biases, dtype=np.double)
    var = num_in_pixels

    # handle affine layer
    for i in range(num_out_pixels):
        tdim= ElinaDim(var)
        linexpr0 = analyzer.generate_linexpr0(weights[i],biases[i],num_in_pixels)
        element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
        var+=1

    dimrem = elina_dimchange_alloc(0,num_in_pixels)
    for i in range(num_in_pixels):
        dimrem.contents.dim[i] = i
    elina_abstract0_remove_dimensions(man, True, element, dimrem)
    elina_dimchange_free(dimrem)

    dims = elina_abstract0_dimension(man,element)


    # get bounds for each output neuron
    bounds = elina_abstract0_to_box(man,element)
    sup = [bounds[i].contents.sup.contents.val.dbl for i in range(num_out_pixels)]
    inf = [bounds[i].contents.inf.contents.val.dbl for i in range(num_out_pixels)]
    myinf.append(inf)
    mysup.append(sup)
    #print("------bounds_layer{}-------".format(layerno))
    #for (low,high) in zip(inf,sup):
    #    print([low,high])

    
    # handle ReLU layer
  #  print(nn.layertypes[layerno])
    if(nn.layertypes[layerno]=='ReLU'):
       element = relu_box_layerwise(man,True,element,0, num_out_pixels)
    nn.ffn_counter+=1

    '''
    if(nn.layertypes[layerno] in ['ReLU', 'Affine']):
        weights = nn.weights[nn.ffn_counter]
        biases = nn.biases[nn.ffn_counter]

        num_out_pixels = len(weights)
        print(num_out_pixels)
        np.ascontiguousarray(weights, dtype=np.double)
        np.ascontiguousarray(biases, dtype=np.double)
        bounds = []
        # handle linear solver
        for j in range(num_out_pixels):
            bounds.append(linear_solver(inf, sup, weights, biases, j, num_out_pixels))
    '''
bounds = elina_abstract0_to_box(man,element)
sup = [bounds[i].contents.sup.contents.val.dbl for i in range(num_out_pixels)]
inf = [bounds[i].contents.inf.contents.val.dbl for i in range(num_out_pixels)]
print("-------box--------")
for (low,high) in zip(inf,sup):
    print([low,high])
#print(sup)
end = time.time()
print("---------gurobi--------")
for i in range (0,10):
    low, high = linear_solver(myinf,mysup,nn.weights,nn.biases,nn.numlayer,i) 
    print([low,high])


#print("Total time: ", end-start)

-------box--------
[-0.0, 3.592151947425326]
[-0.0, 4.1676422932177495]
[3.4132873813532445, 11.980752603776192]
[-0.0, 3.4986134347055806]
[-0.0, 0.0]
[-0.0, 3.936536262931196]
[-0.0, 7.3249877354403035]
[-0.0, 0.0]
[-0.0, 1.749487391987373]
[-0.0, 0.0]
---------gurobi--------
[0.0, 2.7519528964998403]
[0.0, 3.1047626861533795]
[5.332390476830288, 10.047806935549431]
[0.0, 2.520819550080346]
[0.0, -0.0]
[0.0, 2.979485718885269]
[0.46391271575787396, 5.633766481861193]
[0.0, -0.0]
[0.0, 1.2191987768817663]
[0.0, -0.0]


In [169]:


mybounds = elina_abstract0_to_box(man, element)


# handle ReLU layer
if(nn.layertypes[layerno]=='ReLU'):
  element = relu_box_layerwise(man,True,element,0, num_out_pixels)
nn.ffn_counter+=1

mybounds = elina_abstract0_to_box(man, element)

for i in range(num_out_pixels):
   inf = mybounds[i].contents.inf.contents.val.dbl
   sup = mybounds[i].contents.sup.contents.val.dbl
   #print("----boxRelu----")
   print([inf,sup])

[0.1312565894060258, 3.6691012807485746]
[-0.0, 1.111831678621662]
[9.02723227998303, 12.351875707998241]
[4.297852408137126, 6.053015030849855]
[-0.0, 0.0]
[-0.0, 0.0]
[-0.0, 0.0]
[-0.0, 0.0]
[-0.0, 0.28490370048291314]
[-0.0, 0.0]


In [104]:
nn.ffn_counter

2

In [110]:

weights = nn.weights[nn.ffn_counter]
biases = nn.biases[nn.ffn_counter]
dims = elina_abstract0_dimension(man,element)
num_in_pixels = 10


dimadd = elina_dimchange_alloc(0,num_out_pixels)
for i in range(num_out_pixels):
   dimadd.contents.dim[i] = num_in_pixels
elina_abstract0_add_dimensions(man, True, element, dimadd, False)
elina_dimchange_free(dimadd)
np.ascontiguousarray(weights, dtype=np.double)
np.ascontiguousarray(biases, dtype=np.double)
var = num_in_pixels
# handle affine layer
for i in range(num_out_pixels):
   tdim= ElinaDim(var)
   linexpr0 = analyzer.generate_linexpr0(weights[i],biases[i],num_in_pixels)
   element = elina_abstract0_assign_linexpr_array(man, True, element, tdim, linexpr0, 1, None)
   var+=1
dimrem = elina_dimchange_alloc(0,num_in_pixels)
for i in range(num_in_pixels):
   dimrem.contents.dim[i] = i
elina_abstract0_remove_dimensions(man, True, element, dimrem)
elina_dimchange_free(dimrem)

mybounds = elina_abstract0_to_box(man, element)

for i in range(num_out_pixels):
   inf = mybounds[i].contents.inf.contents.val.dbl
   sup = mybounds[i].contents.sup.contents.val.dbl
   #print("----boxRelu----")
   print([inf,sup])

[-10.740727539689498, -7.911915563982961]
[8.947382256398894, 12.519431280353366]
[4.890190259843939, 6.8273583294469224]
[3.7559529899440345, 5.058767263361327]
[-4.084116651199228, -2.9037728824491706]
[-9.162058397233936, -7.315615608231473]
[-10.77053489748994, -8.091917129709474]
[5.186612189361935, 7.495618662730503]
[3.4985434000454623, 5.413885413339844]
[-3.6492257788578204, -2.1179019723370103]


In [93]:
weights = nn.weights[2]
biases = nn.biases[2]
        
num_out_pixels = len(weights)
np.ascontiguousarray(weights, dtype=np.double)
np.ascontiguousarray(biases, dtype=np.double)
        
bounds = []
# handle linear solver
for j in range(10):
    bounds.append(linear_solver(inf, sup, weights, biases, j, 10))
    

bounds = np.array(bounds)
inf, sup = bounds[:,0], bounds[:,1]

TypeError: 'float' object is not subscriptable

In [93]:
inf

[8.2695626691656,
 -1.8492529169204672,
 -8.05148974726694,
 -15.1312405196088,
 -0.24484312579915027,
 -5.84094186415865,
 3.780815886684236,
 2.2538752421523047,
 -1.533703228248571,
 3.5326610494114425]

In [95]:
num_in_pixels

784

In [96]:
set_I = range(10) 
set_J = range(10)

set_optimize_I = [i for i in set_I if inf[i]<0 and sup[i]>0] #boxes optimized by linear program
set_zeros_I = [i for i in set_I if inf[i]<=0 and sup[i]<=0]  #boxes taken to [0,0] by relu 
set_leave_I = [i for i in set_I if inf[i]>0 and sup[i]>0] #boxes left as before relu    

In [97]:
set_optimize_I

[4]

In [ ]:
def linear_solver(inf, sup, weights, biases, j, num_in_pixels):
    
    #j is the index of the current output node optimized

    set_I = range(num_in_pixels) 
    set_J = range(num_in_pixels)
    

    set_optimize_I = [i for i in set_I if inf[i]<0 and sup[i]>0] #boxes optimized by linear program
    set_zeros_I = [i for i in set_I if inf[i]<=0 and sup[i]<=0]  #boxes taken to [0,0] by relu 
    set_leave_I = [i for i in set_I if inf[i]>0 and sup[i]>0] #boxes left as before relu    
    
    lambda_ = [0]*len(set_I)
    for i in set_optimize_I:
        lambda_[i] = sup[i]/(sup[i]-inf[i])


    mu = [0]*len(set_I)
    for i in set_optimize_I:
        mu[i] = (-1)*(sup[i]*inf[i])/(sup[i]-inf[i])

        
    #---------------------------------------
    #MINIMIZATION
    
    opt_model = Model("mip1")
    #opt_model.setParam('OutputFlag', False)
    h_i_vars  = opt_model.addVars(num_in_pixels, lb=inf, ub=sup, name='h')

    relu_h_i_vars  = opt_model.addVars(num_in_pixels, name='relu_h')

        # Set objective sum over i wji*relu_h_is
    objective = grb.quicksum(weights[j,i] * relu_h_i_vars[i] for i in set_I) + biases[j]

        # for minimization
    

    opt_model.setObjective(objective, GRB.MINIMIZE)

    constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.LESS_EQUAL,
                                           rhs=lambda_[i] * h_i_vars[i] + mu[i], name="constraint_{0}".format(i))
                   for i in set_optimize_I}
   
        # >= constraints
    constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=0, name="constraint_{0}".format(i))
                   for i in set_optimize_I}

    # >= constraints
    constraints = {i :opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=h_i_vars[i], 
                                          name="constraint_{0}".format(i)) for i in set_optimize_I}
        
    # == constraints
    constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=h_i_vars[i],name="constraint_{0}".format(i)) 
                   for i in set_leave_I}

    constraints = {i : opt_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=0,name="constraint_{0}".format(i)) 
                  for i in set_zeros_I}

    opt_model.setParam('OutputFlag', False)
    opt_model.optimize()
    #opt_model.computeIIS()
    opt_model.write('out1_min.mps')
    #opt_model.write('out1.ilp')
    h_j_min = opt_model.objVal
    
    #-------------------------------------------------
    #MAXIMIZATION  

    max_model = Model("max")
    
    #opt_model.setParam('OutputFlag', False)
    h_i_vars  = max_model.addVars(num_in_pixels, lb=inf, ub=sup, name='h')

    relu_h_i_vars  = max_model.addVars(num_in_pixels, lb=0.0, ub=GRB.INFINITY, name='relu_h')

        # Set objective sum over i wji*relu_h_is
    objective = grb.quicksum(weights[j,i] * relu_h_i_vars[i]for i in set_I) + biases[j]

        
    

    max_model.setObjective(objective, GRB.MAXIMIZE)

    constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.LESS_EQUAL,
                                           rhs=lambda_[i] * h_i_vars[i] + mu[i], name="constraint_{0}".format(i))
                   for i in set_optimize_I}
    
    # >= constraints
    constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=0, name="constraint_{0}".format(i))
                   for i in set_optimize_I}

    # >= constraints
    constraints = {i :max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.GREATER_EQUAL,rhs=h_i_vars[i], 
                                          name="constraint_{0}".format(i)) for i in set_optimize_I}
  
    # == constraints
    constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=h_i_vars[i],name="constraint_{0}".format(i)) 
                   for i in set_leave_I}


    constraints = {i : max_model.addConstr(lhs=relu_h_i_vars[i],sense=grb.GRB.EQUAL,rhs=0,name="constraint_{0}".format(i)) 
                  for i in set_zeros_I}

    max_model.setParam('OutputFlag',False)
    max_model.optimize()
    #max_model.computeIIS()
    max_model.write('out_max.mps')
    #max_model.write('out1.ilp')
    
    h_j_max = max_model.objVal
    
    return h_j_min, h_j_max